In [16]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools

In [17]:
n = 20
p = 0.1
eta = 0.1

comparison_graph = generateGraphMUN(n, p, eta)
#comparison_graph = generateGraphERO(n, p, eta)

UndefVarError: UndefVarError: generateGraphMUN not defined